# **Deep Learning For Lung Cancer Prediction Based on Transcriptomic Data : a Basic MLP with no Transfer Learning**
> Author : **Aymen MERROUCHE**. <br>
> In this notebook, we implement a basic MLP for our binary classification task with no Transfer Learning.

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import datetime

from utils import *
from train import *
from data_utils import *
from modules.MLP import *
from modules.focal_loss import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# device to use, if cuda available then use cuda else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Working on : ", device)

Working on :  cuda


In [6]:
# load hyperparametrs
# data paths args
with open('./configs/data_paths.yaml', 'r') as stream:
    data_paths_args  = yaml.load(stream,Loader=yaml.Loader)
    
    
# basic cnn args
with open('./configs/mlp.yaml', 'r') as stream:
    mlp_args  = yaml.load(stream,Loader=yaml.Loader)

## **1 - Get The Data :**

In [7]:
%%time
# Getting the data
# dataset
lung_dataset = TranscriptomicVectorsDatasetLung(data_paths_args["path_to_pan_cancer_hdf5_files"])
lung_dataloader_train, lung_dataloader_validation = get_data_loaders(lung_dataset, batch_size_train = mlp_args["batch_size_ft"],\
                                                                             batch_size_validation = mlp_args["batch_size_ft"])

CPU times: user 11.3 s, sys: 281 ms, total: 11.6 s
Wall time: 11.6 s


## **1 - Network, Criterion and Training :**

In [8]:
# network
net = MLP(len(lung_dataset[0][0])).to(device).double()

# loss and optimizer  
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=mlp_args['lr_pt'])

# Logging + Experiment

ignore_keys = {'no_tensorboard'}
# get hyperparameters with values in a dict
hparams = {**mlp_args}
# generate a name for the experiment
expe_name = '_'.join([f"{key}={val}" for key, val in hparams.items()])
print("Experimenting with : \n \t"+expe_name)
# path where to save the model
savepath = Path('/tempory/transcriptomic_data/pre_trained_mlp_checkpt.pt')
# Tensorboard summary writer
if mlp_args['no_tensorboard']:
    writer = None
else:
    writer = SummaryWriter("runs/runs"+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+expe_name)
    
# start the experiment
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, lung_dataloader_train, lung_dataloader_validation, mlp_args['epochs'], writer=writer)
if not mlp_args['no_tensorboard']:
    writer.close()

Epoch 1/50:  20%|██        | 1/5 [00:00<00:00,  6.61it/s, loss=1.5614e-01]

Experimenting with : 
 	epochs=50_batch_size_pt=128_lr_pt=0.001_batch_size_ft=128_lr_ft=0.001_no_tensorboard=True
Training on GPU 



Epoch 1/50: 100%|██████████| 5/5 [00:00<00:00, 12.14it/s, loss=2.3431e-01]


Epoch 1/50, Train Loss: 2.1475e-01, Test Loss: 0.7139
Epoch 1/50, Train Accuracy: 77.21%, Test Accuracy: 59.35%
Epoch 1/50, Train AUC: 84.57%, Test AUC: 67.68%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.95      0.61      0.74       255
      Cancer       0.15      0.67      0.25        27

    accuracy                           0.61       282
   macro avg       0.55      0.64      0.49       282
weighted avg       0.87      0.61      0.69       282



Epoch 2/50: 100%|██████████| 5/5 [00:00<00:00, 16.51it/s, loss=6.2987e-02]


...

Epoch 48/50, Train Loss: 1.5489e-05, Test Loss: 0.0655
Epoch 48/50, Train Accuracy: 100.00%, Test Accuracy: 90.40%
Epoch 48/50, Train AUC: 100.00%, Test AUC: 62.27%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.98      0.94       255
      Cancer       0.40      0.15      0.22        27

    accuracy                           0.90       282
   macro avg       0.66      0.56      0.58       282
weighted avg       0.87      0.90      0.88       282



Epoch 49/50: 100%|██████████| 5/5 [00:00<00:00, 16.57it/s, loss=2.4892e-04]


Epoch 49/50, Train Loss: 2.6381e-05, Test Loss: 0.0670
Epoch 49/50, Train Accuracy: 100.00%, Test Accuracy: 90.67%
Epoch 49/50, Train AUC: 100.00%, Test AUC: 61.86%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.98      0.95       255
      Cancer       0.44      0.15      0.22        27

    accuracy                           0.90       282
   macro avg       0.68      0.56      0.58       282
weighted avg       0.87      0.90      0.88       282



Epoch 50/50: 100%|██████████| 5/5 [00:00<00:00, 16.87it/s, loss=2.4375e-05]


Epoch 50/50, Train Loss: 1.6679e-05, Test Loss: 0.0674
Epoch 50/50, Train Accuracy: 100.00%, Test Accuracy: 90.93%
Epoch 50/50, Train AUC: 100.00%, Test AUC: 61.45%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.98      0.95       255
      Cancer       0.50      0.15      0.23        27

    accuracy                           0.90       282
   macro avg       0.71      0.57      0.59       282
weighted avg       0.88      0.90      0.88       282


Finished.
Best validation AUC: 6.7683e-01
With accuracy: 0.5935496886571249
